<a href="https://colab.research.google.com/github/NikitaSUAI/EmotionRecognition/blob/main/Union_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pickle
import torch
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

# !cp -r '/content/drive/MyDrive/OMG_EMO' 'OMG_EMO'

BASE_PATH = Path("/content/drive/MyDrive/OMG_EMO")

TRAIN_PATH = BASE_PATH / "train_set.csv"
VAL_PATH = BASE_PATH / "val_set.csv"
TEST_PATH = BASE_PATH / "test_set.csv"


Mounted at /content/drive


In [18]:
import pandas as pd

train_df = pd.read_csv(TRAIN_PATH).dropna()
val_df = pd.read_csv(VAL_PATH).dropna()
test_df = pd.read_csv(TEST_PATH).dropna()

In [40]:
import numpy as np

experiments = [
    "FB_NoVAD_mv3_multytask_CE+MSE",
    "opensmile_FNN_multytask_CE+MSE",
    "Fbanks_FNN_multytask_CE+MSE",
    "W2V_FNN_multytask_CE+MSE",
    "TF-IDF_FNN_multytask_CE+MSE",
    "W2V_RNN_multytask_CE+MSE"
]

train_res, test_res = {}, {}
for EXPIREMENT_NAME in experiments:
  with open(f"/content/drive/MyDrive/OMG_EMO/unswers/{EXPIREMENT_NAME}_train.pkl", "rb") as f:
    train_res[EXPIREMENT_NAME] = pickle.load(f)
    train_res[EXPIREMENT_NAME] = {path : np.concatenate((val[0][0].numpy(), val[1][0].numpy(), val[2][0].numpy())) for val, path in zip(*train_res[EXPIREMENT_NAME])}
  with open(f"/content/drive/MyDrive/OMG_EMO/unswers/{EXPIREMENT_NAME}_test.pkl", "rb") as f:
    test_res[EXPIREMENT_NAME] = pickle.load(f)
    test_res[EXPIREMENT_NAME] = {path : np.concatenate((val[0][0].numpy(), val[1][0].numpy(), val[2][0].numpy())) for val, path in zip(*test_res[EXPIREMENT_NAME])}

In [41]:
train_res["FB_NoVAD_mv3_multytask_CE+MSE"]["train/71091df7b_6/utterance_79.wav"]

array([ 0.11388326,  0.06285179, -0.0297657 , -0.36670518, -0.4732749 ,
        0.08678203,  0.2734528 , -0.21933964,  0.54604924], dtype=float32)

In [46]:
feats_val = dict()
feats_train = dict()
for EXPIREMENT_NAME in experiments:
  feats_val[EXPIREMENT_NAME] = []
  for idx, row in val_df.iterrows():
    feats_val[EXPIREMENT_NAME].append(test_res[EXPIREMENT_NAME][row.path])
  feats_val[EXPIREMENT_NAME] = np.stack(feats_val[EXPIREMENT_NAME])

  feats_train[EXPIREMENT_NAME] = []
  for idx, row in pd.concat([train_df, test_df]).iterrows():
    feats_train[EXPIREMENT_NAME].append(train_res[EXPIREMENT_NAME][row.path])
  feats_train[EXPIREMENT_NAME] = np.stack(feats_train[EXPIREMENT_NAME])

In [52]:
np.concatenate((feats_train[EXPIREMENT_NAME], feats_train[EXPIREMENT_NAME]), axis=1).shape

(3087, 18)

In [54]:
train_set = np.concatenate([feats_train[EXPIREMENT_NAME] for EXPIREMENT_NAME in experiments])
test_set = np.concatenate([feats_val[EXPIREMENT_NAME] for EXPIREMENT_NAME in experiments])

In [138]:
experiments = [
    # "FB_NoVAD_mv3_multytask_CE+MSE",
    # "opensmile_FNN_multytask_CE+MSE",
    # "Fbanks_FNN_multytask_CE+MSE",
    "W2V_FNN_multytask_CE+MSE",
    # "TF-IDF_FNN_multytask_CE+MSE",
    "W2V_RNN_multytask_CE+MSE"
]

avg = np.mean(np.stack([feats_val[EXPIREMENT_NAME] for EXPIREMENT_NAME in experiments], axis=0), axis=0)

In [139]:
result = np.stack([np.argmax(avg[:, 0:7], axis=1), avg[:, 7], avg[:, 8]])

In [140]:
from sklearn.metrics import f1_score
from scipy.stats import pearsonr

def ccc(y_true, y_pred, **kwargs):
    true_mean = np.mean(y_true)
    true_variance = np.var(y_true)
    pred_mean = np.mean(y_pred)
    pred_variance = np.var(y_pred)

    rho,_ = pearsonr(y_pred,y_true)

    std_predictions = np.std(y_pred)

    std_gt = np.std(y_true)


    ccc = 2 * rho * std_gt * std_predictions / (
        std_predictions ** 2 + std_gt ** 2 +
        (pred_mean - true_mean) ** 2)

    return float(ccc)

(f1_score(val_df.EmotionMaxVote.to_numpy(), result[0], average='macro'),
 f1_score(val_df.EmotionMaxVote.to_numpy(), result[0], average='micro'), 
 ccc(val_df.valence.to_numpy(), result[1], average='micro'),
 ccc(val_df.arousal.to_numpy(), result[2], average='micro'))

(0.175604562188185,
 0.3293768545994065,
 0.18273164840005143,
 -0.016124844206148497)